In [1]:
!pip install selenium

  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 2.0 MB/s eta 0:00:00a 0:00:01
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.12.14
    Uninstalling certifi-2024.12.14:
      Successfully uninstalled certifi-2024.12.14
  Attempting uninstall: attrs
    Found existing installation: attr

In [22]:
import pandas as pd 
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

driver_path = "/Users/eunahjung/Desktop/chromedriver"

#1. Setting up chromedriver
service = Service(driver_path)
options = Options()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 Chrome/120 Safari/537.36")

driver = webdriver.Chrome(service=service, options=options)

In [ ]:
#2. Getting the LinkedIn URL 
url = 'https://www.linkedin.com/jobs/search/?currentJobId=4364082236&distance=25.0&f_PP=100862922&geoId=103291313&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=R'
driver.get(url)
time.sleep(3)

In [8]:
elem = driver.find_element(By.CLASS_NAME, 'base-card')
elem.text

'Fresh Grad Hire - Machine Learning Engineer (Large Model Applications)\nFresh Grad Hire - Machine Learning Engineer (Large Model Applications)\nFutu Holdings Limited\nHong Kong, Hong Kong SAR\nBe an early applicant\n1 week ago'

In [ ]:
link_elem = elem.find_element(By.CLASS_NAME, "base-card__full-link")
link_elem.get_attribute("href")

'https://hk.linkedin.com/jobs/view/fresh-grad-hire-machine-learning-engineer-large-model-applications-at-futu-holdings-limited-4356429874?position=1&pageNum=0&refId=IE86B6zNk1ZEnpUElC1vZA%3D%3D&trackingId=H7uhxMvwMuucfZ80O3Pq4w%3D%3D'

In [ ]:
elem = driver.find_element(By.CLASS_NAME, 'base-search-card__info')
elem.text

'Fresh Grad Hire - Machine Learning Engineer (Large Model Applications)\nFutu Holdings Limited\nHong Kong, Hong Kong SAR\nBe an early applicant\n1 week ago'

In [23]:
title = elem.find_element(By.CLASS_NAME, 'base-search-card__title')
title.text

'Fresh Grad Hire - Machine Learning Engineer (Large Model Applications)'

In [24]:
sub_title = elem.find_element(By.CLASS_NAME, 'base-search-card__subtitle')
sub_title.text

'Futu Holdings Limited'

In [27]:
from selenium.webdriver.common.keys import Keys

#3. Collecting data
jobs_list = []
page = 1
max_pages = 3

while page <= max_pages:
    print(f"📄 Collecting data from page {page} ...")

    review_elements = driver.find_elements(By.CLASS_NAME, "base-card")
    for r in review_elements:
        try:
            # Company Name
            company_name = r.find_element(By.CLASS_NAME, "base-search-card__subtitle").text

            # Job Title
            title = r.find_element(By.CLASS_NAME, "base-search-card__title").text

            # Posted Date (optional - may not exist on all cards)
            try:
                date = r.find_element(By.CSS_SELECTOR, "time.job-search-card__listdate").get_attribute("datetime")
            except NoSuchElementException:
                date = "N/A"

            # URL (optional - may not exist on all cards)
            try:
                link = r.find_element(By.CLASS_NAME, "base-card__full-link").get_attribute("href")
            except NoSuchElementException:
                link = "N/A"

            jobs_list.append({
                "company": company_name,
                "title": title,
                "date": date,
                "link": link
            })
            print(f"✅ Added: {title}")

        except Exception as e:
            print("❌ Failed with Parsing:", e)
            continue

    # Scroll to the bottom
    html = driver.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.END) # Scrolls to the bottom
    time.sleep(5)
    is_at_bottom = driver.execute_script("""
        return (window.pageYOffset + window.innerHeight) >= (document.documentElement.scrollHeight - 1);
    """)
    if is_at_bottom:
        print("✅ Reached the bottom of the page.")
        break

📄 Collecting data from page 1 ...
✅ Added: Fresh Grad Hire - Machine Learning Engineer (Large Model Applications)
✅ Added: Junior Engineer
✅ Added: Fresh Grad Hire - Client Development Engineer (Market Data)
✅ Added: Data Scientist(Manager level) - VOICE
✅ Added: Machine Learning Engineer
✅ Added: Senior Manager, Data Scientist
✅ Added: AI Engineer
✅ Added: Data Scientist, Risk & Quant Analytics
✅ Added: 后端工程师 (Kotlin) - (HK)
✅ Added: Fresh Grad Hire - Client Developer (Digital Bank)
✅ Added: Data Science Manager
✅ Added: Programmer (C#.net, 5 days’ work, over $40K)
✅ Added: Fresh Grad Hire - System Testing Engineer (OTC Trading Platform)
✅ Added: Graduate Software Engineer
✅ Added: AI Engineer (LLM)
✅ Added: AI Trainer - Hong Kong
✅ Added: Fresh Grad Hire - Backend Developer (Financial Securities System R&D Focus)
✅ Added: Senior Data Scientist (10 months contractor)
✅ Added: Assistant Data Science Manager
✅ Added: Senior Data Scientist, Algorithm (Risk & Fraud)
✅ Added: Researcher
✅ 

In [28]:
jobs_list

[{'company': 'Futu Holdings Limited',
  'title': 'Fresh Grad Hire - Machine Learning Engineer (Large Model Applications)',
  'date': '2026-01-30',
  'link': 'https://hk.linkedin.com/jobs/view/fresh-grad-hire-machine-learning-engineer-large-model-applications-at-futu-holdings-limited-4356429874?position=1&pageNum=0&refId=B9hgQngWT0%2FxSINVrPy2ug%3D%3D&trackingId=G9xJozyNQIaBXWL%2BSj3pYg%3D%3D'},
 {'company': 'First Circle',
  'title': 'Junior Engineer',
  'date': '2026-02-10',
  'link': 'https://hk.linkedin.com/jobs/view/junior-engineer-at-first-circle-4370972305?position=2&pageNum=0&refId=B9hgQngWT0%2FxSINVrPy2ug%3D%3D&trackingId=g3UPrBQXoWFS9qYTWz68Gw%3D%3D'},
 {'company': 'Futu Holdings Limited',
  'title': 'Fresh Grad Hire - Client Development Engineer (Market Data)',
  'date': '2026-02-03',
  'link': 'https://hk.linkedin.com/jobs/view/fresh-grad-hire-client-development-engineer-market-data-at-futu-holdings-limited-4357476265?position=3&pageNum=0&refId=B9hgQngWT0%2FxSINVrPy2ug%3D%3D

In [ ]:
# 5. Output
df = pd.DataFrame(jobs_list)
df.head()
df.to_csv('/Users/eunahjung/Documents/JobScraping/output_with_links.csv', index=False)
